In [5]:
from pprint import pprint
import base64
import json
import mimetypes
import os
import requests
import sys
import pandas as pd
from openai import OpenAI

In [ ]:
master_df = pd.read_csv("../master_list.csv")
master_df.head()

,filepath,xmin,ymin,xmax,ymax,label,ProductType,Weight,HalalStatus,HealthStatus,new_camera,tag
0,IMG_20230428_123528_jpg.rf.5687b7b914f6d9aa98c...,151,42,497,591,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN
1,IMG_20230428_123522_jpg.rf.204ff37f497f2dce442...,88,81,442,567,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN
2,IMG_20230428_123708_jpg.rf.141ecd0cefaea75c0b7...,35,34,492,622,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN
3,IMG_20230428_123521_jpg.rf.1069b402272252862ec...,99,122,428,587,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN
4,IMG_20230428_123659_jpg.rf.5e1b6c4caabe48cf360...,103,17,474,592,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN


In [ ]:
# import time

# # Function to make an API call with retry mechanism
# def make_api_call_with_retry(payload, headers, max_retries=5, backoff_factor=2):
#     retry_count = 0
#     while retry_count < max_retries:
#         try:
#             response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
#             response.raise_for_status()  # Raises HTTPError for bad responses
#             content = response.json().get('choices', [{}])[0].get('message', {}).get('content', '')
#             if content:  # Check if content is not empty
#                 return content
#             else:
#                 raise ValueError("Empty content received")
#         except (requests.exceptions.RequestException, ValueError, KeyError) as e:
#             print(f"Attempt {retry_count + 1} failed: {e}")
#             time.sleep(backoff_factor ** retry_count)  # Exponential backoff
#             retry_count += 1
#     raise Exception("API call failed after maximum number of retries")

    
# # List of image paths (relative or absolute)
# image_paths = master_df['filepath'][287:500]

# # Placeholder for the parsed data
# parsed_data = []

# # Iterate over each image path
# for tmp_img_path in image_paths:
#     image_path = os.path.join("../all_images/", tmp_img_path)
#     base64_image = encode_image(image_path)

#     payload = {
#               "model": "gpt-4-vision-preview",
#               "messages": [
#                 {
#                   "role": "user",
#                   "content": [
#                     {
#                       "type": "text",
#                       "text": """ for this image, can you a prediction for the following three labels?
#                         product type (including Nuts
#                         HoneyOtherSpreads
#                         Babyfood
#                         BiscuitsCrackersCookies
#                         InstantMeals
#                         OtherNoodles
#                         SweetsChocolatesOthers
#                         BeehoonVermicelliMeesua
#                         BabyMilkPowder
#                         Coffee
#                         PotatochipsKeropok
#                         MaternalMilkPowder
#                         OtherDriedFood
#                         Pasta
#                         PotatoSticks
#                         Sugar
#                         CerealBeveragePowder
#                         RolledOatsInstantOatmeal
#                         OtherSauceDressingSoupbasePaste
#                         Kaya
#                         FruitJuice
#                         Book
#                         Flour
#                         InstantNoodles
#                         Salt
#                         SoftDrinks
#                         NutellaChocolate
#                         Sardines
#                         AdultMilk
#                         RiceBrownOthers
#                         Oil
#                         WetWiper
#                         OtherBakingNeeds
#                         CornChip
#                         TeaPowderLeaves
#                         Peanutbutter
#                         BreakfastCerealsCornflakes
#                         CannedPacketCreamersSweet
#                         MiloPowder
#                         ChilliSauce
#                         FlavoredMilk)

#                         image reflection (High, Medium, Low)
#                         image clarity (High, Medium, Low)
#                         prediction confidence for the product type (High, Medium, Low)
                        
#                         only choose from the list of options given in the bracket for each label.

#                         format your answer json format which could be easily converted to dataframe
#                         """
#                     },
#                     {
#                       "type": "image_url",
#                       "image_url": {
#                         "url": f"data:image/jpeg;base64,{base64_image}"
#                       }
#                     }
#                   ]
#                 }
#               ],
#               "max_tokens": 300
#             }
    
    
#     # Using the enhanced function for API calls
#     try:
#         content = make_api_call_with_retry(payload, headers)
        
#         # Parsing the content to extract prediction values
#         # Splitting the content string into lines and then parsing each line
#         lines = content.strip().split('\n')[2:]  # Skipping the header

#         # Parsing the strings to extract the relevant information
#         data_parsed = [line.replace('"', '').strip() for line in lines if ':' in line]
#         data_dict = {item.split(":")[0].strip(): item.split(":")[1].strip().strip(',') for item in data_parsed}

#         # Converting the dictionary into a DataFrame
#         df_from_strings = pd.DataFrame([data_dict])

#         parsed_data.append(df_from_strings)  # Make sure df_from_strings is correctly defined in your parsing logic
#     except Exception as e:
#         print(f"Failed to process image {tmp_img_path}: {e}")




In [10]:
# List of image paths (relative or absolute)
image_paths = master_df['filepath'][287:500]

api_key = ""

# Placeholder for the parsed data
parsed_data = []

# Iterate over each image path
for tmp_img_path in image_paths:
    image_path = os.path.join("../all_images/", tmp_img_path)
    base64_image = encode_image(image_path)
    
    headers = {
          "Content-Type": "application/json",
          "Authorization": f"Bearer {api_key}"
        }

    payload = {
              "model": "gpt-4-vision-preview",
              "messages": [
                {
                  "role": "user",
                  "content": [
                    {
                      "type": "text",
                      "text": """ for this image, can you a prediction for the following three labels?
                        product type (including Nuts
                        HoneyOtherSpreads
                        Babyfood
                        BiscuitsCrackersCookies
                        InstantMeals
                        OtherNoodles
                        SweetsChocolatesOthers
                        BeehoonVermicelliMeesua
                        BabyMilkPowder
                        Coffee
                        PotatochipsKeropok
                        MaternalMilkPowder
                        OtherDriedFood
                        Pasta
                        PotatoSticks
                        Sugar
                        CerealBeveragePowder
                        RolledOatsInstantOatmeal
                        OtherSauceDressingSoupbasePaste
                        Kaya
                        FruitJuice
                        Book
                        Flour
                        InstantNoodles
                        Salt
                        SoftDrinks
                        NutellaChocolate
                        Sardines
                        AdultMilk
                        RiceBrownOthers
                        Oil
                        WetWiper
                        OtherBakingNeeds
                        CornChip
                        TeaPowderLeaves
                        Peanutbutter
                        BreakfastCerealsCornflakes
                        CannedPacketCreamersSweet
                        MiloPowder
                        ChilliSauce
                        FlavoredMilk)

                        image reflection (High, Medium, Low)
                        image clarity (High, Medium, Low)
                        prediction confidence for the product type (High, Medium, Low)
                        
                        only choose from the list of options given in the bracket for each label.

                        format your answer json format which could be easily converted to dataframe
                        """
                    },
                    {
                      "type": "image_url",
                      "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                      }
                    }
                  ]
                }
              ],
              "max_tokens": 300
            }


    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    content = response.json()['choices'][0]['message']['content']

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # Extracting the message content (the table with predictions)
    content = response.json()['choices'][0]['message']['content']

    # Parsing the content to extract prediction values
    # Splitting the content string into lines and then parsing each line
    lines = content.strip().split('\n')[2:]  # Skipping the header

    # Parsing the strings to extract the relevant information
    data_parsed = [line.replace('"', '').strip() for line in lines if ':' in line]
    data_dict = {item.split(":")[0].strip(): item.split(":")[1].strip().strip(',') for item in data_parsed}

    # Converting the dictionary into a DataFrame
    df_from_strings = pd.DataFrame([data_dict])

    parsed_data.append(df_from_strings)


KeyError: 'choices'

In [11]:
# Normalizing column names for consistency across all DataFrames
normalized_dfs = []
# for df in parsed_data:
for i in range(len(parsed_data)):
    df = parsed_data[i]
    if df.shape[1] == 5:
        df = df.iloc[:,1:]
    # Renaming columns to have consistent names across all DataFrames
    df.columns = ['product_type', 'image_reflection', 'image_clarity', 'prediction_confidence']
    normalized_dfs.append(df)

# Combining all DataFrames into a single DataFrame
combined_df = pd.concat(normalized_dfs, ignore_index=True)

# Add img filename
combined_df['img_filename'] = image_paths.tolist()[:len(combined_df)]
combined_df = combined_df[['img_filename', 'product_type', 'image_reflection', 'image_clarity', 'prediction_confidence']]

combined_df

,img_filename,product_type,image_reflection,image_clarity,prediction_confidence
0,IMG_3359_jpeg.rf.77af050e253fe6a423b24a967abd7...,HoneyOtherSpreads,Medium,High,Low
1,IMG_3351_jpeg.rf.30eae972321c3a6d94b756a4d0e6e...,Nuts,Low,High,High
2,IMG_3356_jpeg.rf.312e38d14d49b2812a58961d2cee2...,Nuts,Low,High,High
3,IMG_3426_jpeg.rf.4d3f1c113326946d694c6bb95f85d...,Nuts,Low,High,High
4,IMG_3410_jpeg.rf.349ba3ce0e3121fc627c13474e6d8...,OtherDriedFood,Medium,High,Medium
5,IMG_3354_jpeg.rf.44063d67de857c2180c280b73e3ad...,Nuts,Low,High,High
6,IMG_3396_jpeg.rf.3f5af195e6239773f9120a24341d7...,Nuts,Low,High,High
7,IMG_3397_jpeg.rf.7f0a7cb17e2896819bc0744485b19...,Nuts,Low,High,High
8,IMG_3417_jpeg.rf.47cbea090e8f603f0047c4b603fbc...,Nuts,Low,High,High
9,IMG_3355_jpeg.rf.981e1edd384912ddc34c3b3a9d9b7...,Nuts,Low,High,High


In [13]:
combined_df.to_csv("chatgpt/gpt_rst_286-313.csv")

# Add imgs where big model predicts wrongly

In [24]:
big_model_df = pd.read_csv("big_model_wrong_prediction.csv")
big_model_df.head()

,filepath,Correct_pred_bigmodel
0,output_frame_0255.png,no
1,output_frame_0240.png,no
2,IMG_0753_jpeg.rf.84148cda335656bb00d99e8b0c8ee...,no
3,IMG_0761_jpeg.rf.8262cb50117dceef482c37a3fb70c...,no
4,output_frame_0135.png,no


In [25]:
# List of image paths (relative or absolute)
image_paths = big_model_df['filepath'][:30]

api_key = ""

# Placeholder for the parsed data
parsed_data = []

# Iterate over each image path
for tmp_img_path in image_paths:
    image_path = os.path.join("../all_images/", tmp_img_path)
    base64_image = encode_image(image_path)
    
    headers = {
          "Content-Type": "application/json",
          "Authorization": f"Bearer {api_key}"
        }

    payload = {
              "model": "gpt-4-vision-preview",
              "messages": [
                {
                  "role": "user",
                  "content": [
                    {
                      "type": "text",
                      "text": """ for this image, can you a prediction for the following three labels?
                        product type (including Nuts
                        HoneyOtherSpreads
                        Babyfood
                        BiscuitsCrackersCookies
                        InstantMeals
                        OtherNoodles
                        SweetsChocolatesOthers
                        BeehoonVermicelliMeesua
                        BabyMilkPowder
                        Coffee
                        PotatochipsKeropok
                        MaternalMilkPowder
                        OtherDriedFood
                        Pasta
                        PotatoSticks
                        Sugar
                        CerealBeveragePowder
                        RolledOatsInstantOatmeal
                        OtherSauceDressingSoupbasePaste
                        Kaya
                        FruitJuice
                        Book
                        Flour
                        InstantNoodles
                        Salt
                        SoftDrinks
                        NutellaChocolate
                        Sardines
                        AdultMilk
                        RiceBrownOthers
                        Oil
                        WetWiper
                        OtherBakingNeeds
                        CornChip
                        TeaPowderLeaves
                        Peanutbutter
                        BreakfastCerealsCornflakes
                        CannedPacketCreamersSweet
                        MiloPowder
                        ChilliSauce
                        FlavoredMilk)

                        image reflection (High, Medium, Low)
                        image clarity (High, Medium, Low)
                        prediction confidence for the product type (High, Medium, Low)
                        
                        only choose from the list of options given in the bracket for each label.

                        format your answer json format which could be easily converted to dataframe
                        """
                    },
                    {
                      "type": "image_url",
                      "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                      }
                    }
                  ]
                }
              ],
              "max_tokens": 300
            }


    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    content = response.json()['choices'][0]['message']['content']

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # Extracting the message content (the table with predictions)
    content = response.json()['choices'][0]['message']['content']

    # Parsing the content to extract prediction values
    # Splitting the content string into lines and then parsing each line
    lines = content.strip().split('\n')[2:]  # Skipping the header

    # Parsing the strings to extract the relevant information
    data_parsed = [line.replace('"', '').strip() for line in lines if ':' in line]
    data_dict = {item.split(":")[0].strip(): item.split(":")[1].strip().strip(',') for item in data_parsed}

    # Converting the dictionary into a DataFrame
    df_from_strings = pd.DataFrame([data_dict])

    parsed_data.append(df_from_strings)


In [26]:
# Normalizing column names for consistency across all DataFrames
normalized_dfs = []
# for df in parsed_data:
for i in range(len(parsed_data)):
    df = parsed_data[i]
    if df.shape[1] == 5:
        df = df.iloc[:,1:]
    # Renaming columns to have consistent names across all DataFrames
    df.columns = ['product_type', 'image_reflection', 'image_clarity', 'prediction_confidence']
    normalized_dfs.append(df)

# Combining all DataFrames into a single DataFrame
combined_df = pd.concat(normalized_dfs, ignore_index=True)

# Add img filename
combined_df['img_filename'] = image_paths.tolist()[:len(combined_df)]
combined_df = combined_df[['img_filename', 'product_type', 'image_reflection', 'image_clarity', 'prediction_confidence']]

combined_df

,img_filename,product_type,image_reflection,image_clarity,prediction_confidence
0,output_frame_0255.png,PotatochipsKeropok,Low,Medium,High
1,output_frame_0240.png,Instant Noodles,Low,High,High
2,IMG_0753_jpeg.rf.84148cda335656bb00d99e8b0c8ee...,Instant Noodles,Medium,High,High
3,IMG_0761_jpeg.rf.8262cb50117dceef482c37a3fb70c...,Instant Noodles,Low,High,High
4,output_frame_0135.png,Babyfood,Low,High,Medium
5,output_frame_0263.png,PotatoSticks,Low,Medium,High
6,output_frame_0249.png,PotatochipsKeropok,Low,Medium,High
7,output_frame_0222.png,InstantNoodles,Low,Low,Low
8,output_frame_0016.png,Book,Low,Medium,High
9,output_frame_0077.png,OtherDriedFood,Medium,Medium,High


In [27]:
combined_df.to_csv("chatgpt/gptpred_bigmodelwrong.csv")

# Combine all gpt predictions

In [28]:
import pandas as pd
import glob

In [29]:
path = 'chatgpt' # Update this to your folder's path
all_files = glob.glob(path + "/*.csv")
all_files

['chatgpt/gpt_rst_116-160.csv',
 'chatgpt/gpt_rst_286-313.csv',
 'chatgpt/gpt_rst_161-197.csv',
 'chatgpt/gpt_rst_231-246.csv',
 'chatgpt/gpt_rst_246-272.csv',
 'chatgpt/gptpred_bigmodelwrong.csv',
 'chatgpt/gpt_rst_0-115.csv',
 'chatgpt/gpt_rst_272-286.csv',
 'chatgpt/gpt_rst_198-230.csv']

In [30]:
all_dataframes = [] # A list to store each dataframe

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    all_dataframes.append(df)

combined_df = pd.concat(all_dataframes, axis=0, ignore_index=True)


In [31]:
final_df = combined_df.drop_duplicates()
final_df = final_df.iloc[:,1:]
final_df.head()

,img_filename,product_type,image_reflection,image_clarity,prediction_confidence
0,IMG_0184_jpeg.rf.446d62343e17b8ef909b6f0ff4530...,OtherNoodles,Medium,High,Low
1,IMG_0118_jpeg.rf.305bb0eadb135b869c11332c7173a...,OtherSauceDressingSoupbasePaste,Low,High,High
2,IMG_0096_jpeg.rf.4b1ab7e2cd4c285bd9e74d1ec1ab0...,Book,Low,Medium,Medium
3,IMG_0105_jpeg.rf.62a7a2c99f2d635525abcca89cb9f...,OtherSauceDressingSoupbasePaste,Low,Medium,Low
4,IMG_0124_jpeg.rf.3d66e8ddcd7d9d088875a3e2cf365...,OtherDriedFood,Low,High,High


In [32]:
final_df.shape

(350, 5)

In [33]:
final_df.to_csv("chatgpt_prediction.csv")